In [ ]:
import sys
from pathlib import Path
sys.path.insert(1, Path.cwd().parent.as_posix())

from matplotlib import pyplot as plt
import numpy as np
from typing import TypedDict
from utils.oscilloscope import Oscilloscope, plot_data
from rberga06.phylab import Datum as d

In [ ]:
class DataSet(TypedDict):
    pass

In [6]:
all_data_day1 = [Oscilloscope.Elettr.load_data(i, dir="data/day1", plot_dir="elab/day1") for i in range(51)]
all_data_day2 = [Oscilloscope.Elettr.load_data(i, dir="data/day2", plot_dir="elab/day2") for i in range(27)]

In [ ]:
dati1 = {
    "freq": np.array([1., ])
}